# MLE

In situations where parameters are unknown, as exemplified in linear regression, the necessity to estimate them becomes apparent. Estimation is statistics, used to determine the truth value of an object or function that may be found within a population. Consequently, using a sample of the population may help with determining this truth value.

<p align="center">
  <img src="https://miro.medium.com/v2/resize:fit:497/1*QGIqfYif8nYpULr2siDDNw.png" />
</p>

Suppose given data: $X_1...X_n$ random sample from distribution depending on parameters $θ=(θ_1, ..., θ_m) $

```{admonition} Question
:class: important

How can θ be estimated?
```

## Maximum Likelihood Estimation
The most common method for estimating parameters in a parametric model is the **maximum likelihood** method. Let $X_1, \ldots, X_n$ be IID with PDF $f(x;\theta)$.

The likelihood function is defined by:

$$
L(\theta) = \prod_{i=1}^{n} f(x_i;\theta)
$$

The log-likelihood function is defined by:

$$
\ln(\theta) = \log L(\theta)
$$
The likelihood function is just the joint density of the data, except that we treat it as a function of the parameter $\theta$. Thus, $L: \theta \to [0, \infty)$. The likelihood function is not a density function; in general, it is not true that $L(\theta)$ integrates to 1 (with respect to $\theta$).

The maximum likelihood estimator (MLE), denoted by $\hat{\theta}_n$, is the value of $\theta$ that maximizes $\ln(\theta)$.

```{admonition} What is the first step with MLE?
:class: dropdown

Identify which parametric class of distributions is generating the data.
Each such class is a family of distributions indexed by a finite number of parameters.
```

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import bernoulli
from scipy.optimize import minimize
from ipywidgets import interact, widgets, Layout

def likelihood_function(p, n, sum_x):
    return p**sum_x * (1-p)**(n-sum_x)

def negative_log_likelihood(p, n, sum_x):
    return -likelihood_function(p, n, sum_x)

def update_plot(n, sum_x):
    probability = sum_x / n
    p_values = np.linspace(0, 1, 100)
    likelihood_values = likelihood_function(p_values, n, sum_x)

    plt.figure(figsize=(8, 6))
    plt.plot(p_values, likelihood_values, label='Likelihood Function', linewidth=2, color='blue')
    plt.axvline(probability, color='red', linestyle='--', label=f'Current Probability: {probability:.3f}', linewidth=2)
    plt.xlabel('Probability (p)', fontsize=14)
    plt.ylabel('Likelihood', fontsize=14)
    plt.title(f'Likelihood Function for Bernoulli Distribution\nwith n = {n} and ∑(Xi) = {sum_x}', fontsize=16)
    plt.legend(fontsize=12)
    plt.grid(True, linestyle='--', alpha=0.7)
    plt.show()

style = {'description_width': 'initial'}
layout = Layout(display='flex', justify_content='center', width='50%')
n_slider = widgets.IntSlider(value=20, min=1, max=100, step=1, description='Sample Size (n)', style=style, layout=layout)
sum_x_slider = widgets.IntSlider(value=12, min=0, max=20, step=1, description='∑(Xi)', style=style, layout=layout)

interact(update_plot, n=n_slider, sum_x=sum_x_slider)


interactive(children=(IntSlider(value=20, description='Sample Size (n)', layout=Layout(display='flex', justify…

<function __main__.update_plot(n, sum_x)>

The maximum of $L_n(0)$ occurs at the same place as the maximum of $l_n(0)$, so maximizing the log-likelihood leads to the same answer as maximizing the likelihood. Often, it is easier to work with the log-likelihood.


```{admonition} Remark
:class: dropdown

If we multiply $L_n(0)$ by any positive constant $c$ (not depending on $0$), then this will not change the Maximum Likelihood Estimation (MLE). Hence, constants in the likelihood function are often dropped.
```

## Example: MLE for Bernoulli Distribution
Suppose that $X_1, \ldots, X_n \sim \text{Bernoulli}(p)$. The probability function is $f(x; p) = p^x(1 - p)^{1-x}$ for $x = 0, 1$. The unknown parameter is $p$. Then,
$$
\mathcal{C}(p) = \prod_{i=1}^{n} f(x_i, p) = \prod_{i=1}^{n} p^{x_i}(1 - p)^{1-x_i} = p^{S}(1 - p)^{n-S}.
$$
where $S = \sum_{i=1} X_i$. Hence,
$$
l_n(p) = S\log(p) + (n - S)\log(1 - p).
$$

<span style="display:none" id="quiz_derivative">W3sicXVlc3Rpb24iOiAiVGFraW5nIHRoZSBkZXJpdmF0aXZlIG9mICRsX24ocCkkIGFuZCBzZXR0aW5nIGl0IHRvIDAsIHdoYXQgd291bGQgYmUgdGhlIE1MRSAoJFxcaGF0IHBfbiQpPyIsICJ0eXBlIjogIm1hbnlfY2hvaWNlIiwgImFuc3dlcnMiOiBbeyJhbnN3ZXIiOiAiJFMvbiQiLCAiY29ycmVjdCI6IHRydWUsICJmZWVkYmFjayI6ICJDb3JyZWN0ISBUaGUgTWF4aW11bSBMaWtlbGlob29kIEVzdGltYXRpb24gZm9yICRcXGhhdCBwX24kIGlzIG9idGFpbmVkIGJ5IHNvbHZpbmcgJFxcZnJhY3tkfXtkcH0gbF9uKHApID0gMCQsIHJlc3VsdGluZyBpbiAkXFxoYXQgcF9uID0gXFxmcmFje1N9e259JC4ifSwgeyJhbnN3ZXIiOiAiJFxcZnJhY3tufXtTfSQiLCAiY29ycmVjdCI6IGZhbHNlLCAiZmVlZGJhY2siOiAiSW5jb3JyZWN0LiJ9LCB7ImFuc3dlciI6ICIkbi9TJCIsICJjb3JyZWN0IjogZmFsc2UsICJmZWVkYmFjayI6ICJJbmNvcnJlY3QuIn0sIHsiYW5zd2VyIjogIiRTK24kIiwgImNvcnJlY3QiOiBmYWxzZSwgImZlZWRiYWNrIjogIkluY29ycmVjdC4ifV19XQ==</span>

In [2]:
from jupyterquiz import display_quiz
display_quiz("#quiz_derivative")

<IPython.core.display.Javascript object>

## Example: Normal Distribution
Let $X_1, \ldots, X_n \sim \mathcal{N}(\mu, \sigma^2)$. The parameter is $\theta = (\mu, \sigma)$, and the likelihood function (ignoring some constants) is:
$$
L_n(\mu, \sigma) = \prod_{i=1}^{n} \frac{1}{\sigma} \exp\left(-\frac{1}{2\sigma^2}(X_i - \mu)^2\right)
= \sigma^{-n} \exp\left(-\frac{1}{2\sigma^2} \sum_{i=1}^{n}(X_i - \mu)^2\right)
= \sigma^{-n} \exp\left(-\frac{nS^2}{2\sigma^2}\right) \exp\left(-\frac{n(X - \mu)^2}{2\sigma^2}\right),
$$
$$
where X = \frac{1}{n} \sum_{i=1}^{n} X_i is the sample mean, and S^2 = \frac{1}{n} \sum_{i=1}^{n}(X_i - X)^2.
The last equality above follows from the fact that \sum_{i}(X_i -\mu)^2 = nS^2 + n(X -\mu)^2, which can be verified by writing \sum_{i}(X_i - \mu)^2 = \sum_{i}(X_i - X + X - \mu)^2 and then expanding the square.
The log-likelihood is given by:
$$
$$
l_n(\mu, \sigma) = -n\log(\sigma) - \frac{nS^2}{2\sigma^2} - \frac{n(X - \mu)^2}{2\sigma^2}.
$$
Solving the equations $\frac{\partial l_n(\mu, \sigma)}{\partial \mu} = 0$ and $\frac{\partial \ln(\mu, \sigma)}{\partial \sigma} = 0$, we conclude that $\hat{\mu} = X$ and $\hat{\sigma} = S$. It can be verified that these are indeed global maxima of the likelihood.


Suppose the weights of randomly selected American female college students are normally distributed with an unknown mean $\mu$ and standard deviation $\sigma$. A random sample of 10 American female college students yielded the following weights (in pounds):

$$
115, 122, 130, 127, 149, 160, 152, 138, 149, 180
$$


<span style="display:none" id="quiz_normal">W3sicXVlc3Rpb24iOiAiV2hhdCB3b3VsZCBiZSB0aGUgbWF4aW11bSBsaWtlbGlob29kIGVzdGltYXRvciBvZiAkXFxtdSQ/IiwgInR5cGUiOiAibnVtZXJpYyIsICJhbnN3ZXJzIjogW3sidHlwZSI6ICJ2YWx1ZSIsICJjb3JyZWN0IjogZmFsc2UsICJmZWVkYmFjayI6ICJJbmNvcnJlY3QhIC4ifSwgeyJ0eXBlIjogInZhbHVlIiwgInZhbHVlIjogIjE0Mi4yIiwgImNvcnJlY3QiOiB0cnVlLCAiZmVlZGJhY2siOiAiQ29ycmVjdCEifV19XQ==</span>

In [3]:
from jupyterquiz import display_quiz
display_quiz("#quiz_normal")

<IPython.core.display.Javascript object>

## Example: Uniform distribution
Here is an example that many people find confusing. Let $X_1, \ldots, X_n \sim \text{Unif}(0, \theta)$. Recall that
$$
 f(x; \theta) = \begin{cases} \frac{1}{\theta}, & 0 \leq x \leq \theta \\ 0, & \text{otherwise} \end{cases}.
$$
$$
Consider a fixed value of \theta. Suppose \theta < X_i for some i. Then, f(X_i; \theta) = 0, and hence L_n(\theta) = \prod_{i} f(X_i; \theta) = 0. It follows that L_n(\theta) = 0 if any X_i > \theta. Therefore, L_n(\theta) = 0 if \theta < X_{(n)} where X_{(n)} = \max\{X_1, \ldots, X_n\}. Now consider any \theta \geq X_{(n)}. For every X_i, we then have that f(X_i; \theta) = \frac{1}{\theta}, so that L_n(\theta) = \prod_{i} f(X_i; \theta) = \theta^{-n}. In conclusion,
$$
$$
 L_n(\theta) = \begin{cases} \theta^{-n}, & \theta \geq X_{(n)} \\ 0, & \theta < X_{(n)} \end{cases}.
$$
Now $L_n(\theta)$ is strictly decreasing over the interval $[X_{(n)}, \infty)$. Hence, $\hat{\theta}_n = X_{(n)}$.


## Properties of MLE


**Equivariance**(or function invariance) states that if you have the MLE ($\hat\theta$) for a parameter $\theta$, and any function $g$ applied to that estimator $g(\hat\theta)$, then $g(\hat\theta)$ becomes the Maximum Likelihood Estimator for $g(\theta)$.

Let, $X_1, \ldots, X_n$ be independent and identically distributed random variables following the Exponential$(\lambda)$. The Probability Density Function (PDF) is given by $f(x) = \lambda e^{-\lambda x}$, $x>0$.
$\text{MLE } \; \hat\lambda = \frac{n}{{X_1 + \ldots + X_n}}$



The MLE is **consistent**: $\hat{\theta}_n \xrightarrow{P} \theta$, where $\theta$ denotes the true value of the parameter $\theta$.


The MLE is **Asymptotically Normal**: $(\hat{\theta}_n - \theta) / \text{se}_n \xrightarrow{} N(0, 1)$; also, the estimated standard error $\text{se}_n$ can often be computed analytically.


<span style="display:none" id="quiz_equivariance">W3sicXVlc3Rpb24iOiAiV2hhdCBpcyB0aGUgTUxFIG9mICRnKFxcbGFtYmRhKSA9IGVeXFxsYW1iZGEkPyIsICJ0eXBlIjogIm1hbnlfY2hvaWNlIiwgImFuc3dlcnMiOiBbeyJhbnN3ZXIiOiAiJGVee1xcaGF0XFxsYW1iZGF9JCIsICJjb3JyZWN0IjogdHJ1ZSwgImZlZWRiYWNrIjogIkNvcnJlY3QhIFRoZSBNTEUgb2YgJGcoXFxsYW1iZGEpJCBpcyAkZV57XFxoYXRcXGxhbWJkYX0kLCB3aGVyZSAkXFxoYXRcXGxhbWJkYSQgaXMgdGhlIE1MRSBvZiAkXFxsYW1iZGEkLiJ9LCB7ImFuc3dlciI6ICIkZV57KG4vWF8xICsgXFxsZG90cyArIFhfbil9JCIsICJjb3JyZWN0IjogdHJ1ZSwgImZlZWRiYWNrIjogIkNvcnJlY3QuIn0sIHsiYW5zd2VyIjogIiRlXnsoWF8xICsgXFxsZG90cyArIFhfbikvbn0kIiwgImNvcnJlY3QiOiBmYWxzZSwgImZlZWRiYWNrIjogIkluY29ycmVjdC4ifSwgeyJhbnN3ZXIiOiAiJGVee259JCIsICJjb3JyZWN0IjogZmFsc2UsICJmZWVkYmFjayI6ICJJbmNvcnJlY3QuIn1dfV0=</span>

In [4]:
from jupyterquiz import display_quiz
display_quiz("#quiz_equivariance")

<IPython.core.display.Javascript object>